In [6]:
import numpy as np
import numpy.typing as npt
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider, FloatSlider

# Pulse Compression

<!-- - We have two goals that are opposed to each other: good range resolution and high SNR
- Why are they opposed? Bam! Show the plot on the Gaussian pulse and it's transform
- In general, short duration pulses help localize, but long pulses allow you to put more energy on the target
- Short pulses help localize because short pulses have high bandwidth, so can we get higher bandwidth while keeping a longer pulse? Yes! -->

In radar systems, achieving high performance often involves balancing conflicting requirements. Two such requirements are having a high signal-to-noise ratio (SNR), and having fine range resolution.

## Signal-to-Noise Ratio (SNR)
SNR is the ratio between the power of the recieved target signal and the noise power. For now, any recieved noise is assumed to be additive noise that follows a Gaussian distribution. 

BLAH BALH BLAH

In [13]:
def sin_pulse(
    t: npt.NDArray, 
    t0: float, 
    f: float, 
    A: float,
    T: float,
) -> npt.NDArray:
    """
    Generate an LFM chirp signal.

    Parameters
    ----------
    t: `[N,] float`
        Time samples

    t0: `float`
        Start time of the chirp

    f0: `float > 0`
        Start frequency of the chirp (Hz)

    k: `float`
        Chirp rate (Hz/s)

    T: `float > 0`
        Pulse duration (seconds)

    Returns
    -------
    signal: `[N,] float`
        LFM chirp signal
    """
    # Initialize the chirp signal with zeros
    t0 -= T/2
    signal = np.zeros_like(t, dtype=float)
    
    # Determine the times within the chirp duration
    mask = (t >= t0) & (t <= t0 + T)
    t_within = t[mask] - t0  # Time relative to the chirp start
    
    # Compute the instantaneous phase for the chirp
    phase = 2 * np.pi * f * t_within
    
    # Set the chirp signal for the valid times
    signal[mask] = A * np.cos(phase)
    
    return signal


def snr_demo(amplitude=1, sigma=1):
    t = np.linspace(-10, 10, 1000)

    signal = sin_pulse(t, 0, 3, amplitude, 1)
    noise = np.random.normal(0, sigma, len(t))
    # Create the plot
    Fig = plt.figure(figsize=(10, 8))
    

    plt.plot(t, signal + noise)
    plt.title("Sine + Noise")
    plt.xlabel("t")
    plt.ylabel("Amplitude")
    plt.ylim((-5, 5))
    plt.grid()
    
    # # Subplot 2: Convolution value
    # Ax2 = plt.subplot(2, 1, 2)
    # Ax2.plot(freqs, g_freq)
    # Ax2.set_title("Frequency Domain")
    # Ax2.set_ylabel("Amplitude")
    # Ax2.set_ylim((-2, 2))
    # Ax2.grid()
    
    plt.tight_layout()


In [ ]:
# Interactive slider for waveform offset
interact(
    snr_demo,
    amplitude=FloatSlider(value=1, min=0.1, max=3, step=0.1, description="Amplitude"),
    sigma=FloatSlider(value=0.5, min=0.05, max=2, step=0.1, description="Noise"),
)

In [9]:
def gauss_pulse(
    t: npt.NDArray, 
    t_center: float, 
    sigma: float,
) -> tuple[npt.NDArray, npt.NDArray]:
    """
    Generate an LFM chirp signal

    Parameters
    ----------
    t: `[N,] float`
        Time samples

    t_center: `float`
        Center of Gaussian

    sigma: `float`
        Standard deviation

    Returns
    -------
    g_time: `[N,] float`
        Gaussian pulse in time domain

    g_freq: `[N,] float`
        Gaussian pulse in frequency domain
    """
    g_time = np.exp(-((t - t_center) ** 2) / (2 * sigma ** 2))
    freqs = np.fft.fftfreq(len(t), d=(t[1] - t[0]))  # Compute frequency axis
    # g_freq = sigma * np.sqrt(2 * np.pi) * np.exp(-2 * (np.pi * sigma * freqs) ** 2)
    g_freq = np.exp(-2 * (np.pi * sigma * freqs) ** 2)

    
    return g_time, g_freq



def fourier_tradeoff_demo(sigma=1):
    t = np.linspace(-10, 10, 1000)
    freqs = np.fft.fftfreq(len(t), d=(t[1] - t[0]))  # Compute frequency axis

    g_time, g_freq = gauss_pulse(t, 0, sigma)
    # Create the plot
    Fig = plt.figure(figsize=(10, 8))
    
    # Subplot 1: Waveforms
    Ax1 = plt.subplot(2, 1, 1)
    Ax1.plot(t, g_time)
    Ax1.set_title("Time Domain")
    Ax1.set_xlabel("t")
    Ax1.set_ylabel("Amplitude")
    Ax1.set_ylim((-2, 2))
    Ax1.grid()
    
    # Subplot 2: Convolution value
    Ax2 = plt.subplot(2, 1, 2)
    Ax2.plot(freqs, g_freq)
    Ax2.set_title("Frequency Domain")
    Ax2.set_ylabel("Amplitude")
    Ax2.set_ylim((-2, 2))
    Ax2.grid()
    
    plt.tight_layout()


In [ ]:
# Interactive slider for waveform offset
interact(
    fourier_tradeoff_demo,
    sigma=FloatSlider(value=0.5, min=0.05, max=1, step=0.01, description="Sigma"),
)

In [ ]:
def lfm_chirp(
    t: npt.NDArray, 
    t0: float, 
    f0: float, 
    k: float, 
    T: float,
) -> npt.NDArray:
    """
    Generate an LFM chirp signal.

    Parameters
    ----------
    t: `[N,] float`
        Time samples

    t0: `float`
        Start time of the chirp

    f0: `float > 0`
        Start frequency of the chirp (Hz)

    k: `float`
        Chirp rate (Hz/s)

    T: `float > 0`
        Pulse duration (seconds)

    Returns
    -------
    chirp_signal: `[N,] float`
        LFM chirp signal
    """
    # Initialize the chirp signal with zeros
    t0 -= T/2
    chirp_signal = np.zeros_like(t, dtype=float)
    
    # Determine the times within the chirp duration
    mask = (t >= t0) & (t <= t0 + T)
    t_within = t[mask] - t0  # Time relative to the chirp start
    
    # Compute the instantaneous phase for the chirp
    phase = 2 * np.pi * (f0 * t_within + 0.5 * k * t_within**2)
    
    # Set the chirp signal for the valid times
    chirp_signal[mask] = np.cos(phase)
    
    return chirp_signal


def generate_overlap():
    x = np.linspace(-10, 10, 1000)
    waveform1 = lfm_chirp(x, 0, 1, 2, 2)
    # waveform1 = np.exp(-x**2 / 4)  # Gaussian function
    # returned1 = np.heaviside(x + 2, 1) - np.heaviside(x - 2, 1)  # Rectangular pulse
    returned1 = 0.5 * lfm_chirp(x, 0, 1, 2, 2)
    
    # Shift the second waveform
    offset = np.linspace(-500, 500, 1000)
    separation = np.arange(0, 8, 0.1)
    convolution = np.zeros((1000, len(separation)))
    for i, shift in enumerate(offset):
        for j, sep in enumerate(separation):
            returned1_shifted = np.roll(returned1, int(shift))
            waveform2 = lfm_chirp(x - sep, 0, 1, 2, 2)
            overlap = sum((waveform1 + waveform2) * returned1_shifted) / len(x)
            convolution[i, j] = overlap
    
    # Compute the overlap integral (convolution at this step)
    return convolution

convolution = generate_overlap()

def convolution_demo(slider=0, separation=0.1):
    """
    Interactive demonstration of convolution.
    offset: Amount to slide the second waveform relative to the first.
    """
    offset = slider - 500
    # Define the waveforms
    x = np.linspace(-10, 10, 1000)
    waveform1 = lfm_chirp(x, 0, 1, 2, 2)
    waveform2 = lfm_chirp(x - separation, 0, 1, 2, 2)
    # waveform1 = np.exp(-x**2 / 4)  # Gaussian function
    # waveform2 = np.heaviside(x + 2, 1) - np.heaviside(x - 2, 1)  # Rectangular pulse
    returned1 = 0.5 * lfm_chirp(x, 0, 1, 2, 2)
    
    # Shift the second waveform
    returned1_shifted = np.roll(returned1, offset)
    
    # Create the plot
    plt.figure(figsize=(10, 8))
    
    # Subplot 1: Waveforms
    plt.subplot(2, 1, 1)
    plt.plot(x, waveform1 + waveform2, label="Template")
    plt.plot(x, returned1_shifted, label="Return Signal")
    plt.title("Return Signal and Template")
    plt.xlabel("x")
    plt.ylabel("Amplitude")
    plt.ylim((-2, 2))
    plt.legend()
    plt.grid()
    
    # Subplot 2: Convolution value
    plt.subplot(2, 1, 2)
    plt.plot(x[0:slider], convolution[0:slider, int(separation * 10)])
    plt.title("Combined Output")
    plt.ylabel("Amplitude")
    plt.xlim((-10, 10))
    plt.ylim((-0.05, 0.05))
    plt.grid()
    
    plt.tight_layout()
    plt.show()

plt.close('all')

# Interactive slider for waveform offset
interact(
    convolution_demo,
    slider=IntSlider(value=0, min=50, max=950, step=1, description="Offset"),
    separation=FloatSlider(value=1, min=0, max=7.9, step=0.1, description="Separation"),
)
